In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import math
from catboost import CatBoostRegressor
from sklearn import ensemble
from sklearn.preprocessing import RobustScaler, StandardScaler
from xgboost.sklearn import XGBClassifier, XGBRegressor
import pickle

### data precess

In [35]:
def data_process(train_data, inference_data ,feature_columns, treatment_columns_category, treatment_columns_common, outcome_column, treatment_columns_continuous):
    # 众数 补充缺失值
    train = pd.concat([train_data,inference_data])
    train= train[feature_columns]
    for col in feature_columns:
        d = dict(train[col].value_counts())
        train[col] = train[col].fillna(sorted(d.items(), key=lambda x: x[1], reverse=True)[0][0])
        
        train_data[col] = train_data[col].fillna(sorted(d.items(), key=lambda x: x[1], reverse=True)[0][0])
        inference_data[col] = inference_data[col].fillna(sorted(d.items(), key=lambda x: x[1], reverse=True)[0][0])

    treatment_columns_category_dict = dict()
    for col in treatment_columns_category:
        label_encoder = LabelEncoder().fit(train[col])
        temp_dict = dict(
            zip(range(0, len(label_encoder.classes_)), label_encoder.classes_))
        treatment_columns_category_dict[col] = temp_dict
        train[col] = label_encoder.transform(train[col])
        
        train_data[col] = label_encoder.transform(train_data[col])
        inference_data[col] = label_encoder.transform(inference_data[col])

    for col in feature_columns:
        if train[col].dtype == 'object':
            label_encoder = LabelEncoder().fit(train[col])
            train[col] = label_encoder.transform(train[col])

            train_data[col] = label_encoder.transform(train_data[col])
            inference_data[col] = label_encoder.transform(inference_data[col])

    xgb = XGBRegressor()
    xgb.fit(train_data[feature_columns], train_data[outcome_column])
    
    inference_data["outcome1"] = xgb.predict(inference_data[feature_columns])

    return treatment_columns_category_dict, inference_data, xgb

mock data

In [36]:
train = pd.read_csv('train.csv')
train = train[['V_0', 'V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9',
              'V_10', 'treatment', 'outcome']]
train['user_id']=train.index

feature_columns=['V_0', 'V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9',
       'V_10', 'treatment']
# treatment_columns_continuous=['V_6', 'V_7']
treatment_columns_category=['V_10', 'treatment']
treatment_columns_common=['V_6','treatment']
treatment_change_value=0.2

outcome_column=["outcome"]
treatment_columns_continuous={
  "V_6": [None, 10, 1],
  "V_7": [0.3, None, 0]
}

train_data=train[0:30000]
inference_data=train[30000:]


In [37]:
treatment_columns_category_dict, inference_data,xgb = data_process(train_data, inference_data ,feature_columns, treatment_columns_category, treatment_columns_common,
             outcome_column, treatment_columns_continuous)

C:\temp\ipykernel_28124\886744685.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[col] = train_data[col].fillna(sorted(d.items(), key=lambda x: x[1], reverse=True)[0][0])
C:\temp\ipykernel_28124\886744685.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inference_data[col] = inference_data[col].fillna(sorted(d.items(), key=lambda x: x[1], reverse=True)[0][0])
C:\temp\ipykernel_28124\886744685.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

C:\temp\ipykernel_28124\886744685.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inference_data["outcome1"] = xgb.predict(inference_data[feature_columns])


In [38]:
treatment_columns_category_dict

{'V_10': {0: 'no', 1: 'unknown', 2: 'yes'}, 'treatment': {0: 0, 1: 1, 2: 2}}

In [39]:
inference_data

,V_0,V_1,V_2,V_3,V_4,V_5,V_6,V_7,V_8,V_9,V_10,treatment,outcome,user_id,outcome1
30000,0.874959,2.330671,1.0,1.397910,1.684222,1.0,-36.4,4.857,0,999,0,0,-9.186014,30000,-15.059911
30001,-0.403105,7.068369,1.0,-1.071795,-1.079318,2.0,-41.8,4.864,0,999,0,2,-14.607914,30001,-15.496181
30002,1.072863,5.101955,0.0,0.430208,-4.575206,1.0,-36.1,4.964,0,999,0,2,-15.362503,30002,-11.112887
30003,2.573898,3.126618,1.0,-0.043587,4.602418,1.0,-41.8,4.947,0,999,2,2,-22.026120,30003,-19.675625
30004,0.131148,1.296791,0.0,-0.020357,5.975856,0.0,-42.0,4.021,0,999,0,0,-34.461070,30004,-18.190821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36183,0.826893,4.331358,0.0,-1.551377,-2.904315,0.0,-46.2,1.250,0,999,1,2,20.512318,36183,10.120279
36184,0.631707,1.383925,0.0,-0.411007,4.483282,1.0,-42.7,4.962,0,999,0,0,-3.178066,36184,-13.517050
36185,-0.072426,4.918745,0.0,-0.351575,-4.926085,1.0,-36.1,4.963,0,999,0,0,-43.733190,36185,-28.395004
36186,-1.029582,4.633457,0.0,-0.496135,3.944163,1.0,-46.2,1.334,0,999,0,2,12.905083,36186,13.861798


In [70]:
def inference(train, treatment_columns_category_dict, xgb, feature_columns,
              treatment_columns_category, treatment_columns_common, outcome_column, treatment_columns_continuous):
    outcome_pd = pd.DataFrame()
    outcome_pd['outcome'] = xgb.predict(train[feature_columns])
    result = dict()

    # 基于连续型变量的处理
    treatment_cont_common = []
    for col, limit_value in treatment_columns_continuous.items():
        if col in treatment_columns_common:
            treatment_cont_common.append(col)
        else:
            train_temp = train.copy(deep=True)
            if limit_value[2] == 0:
                if limit_value[0] != None:
                    train_temp[col] = train_temp[col].apply(
                        lambda x: x*(1-treatment_change_value) if x*(1-treatment_change_value) > limit_value[0] else x)
                if limit_value[1] != None:
                    train_temp[col] = train_temp[col].apply(
                        lambda x: x*(1-treatment_change_value) if x*(1-treatment_change_value) < limit_value[1] else x)
            if limit_value[2] == 1:
                if limit_value[0] != None:
                    train_temp[col] = train_temp[col].apply(
                        lambda x: x*(1+treatment_change_value) if x*(1+treatment_change_value) > limit_value[0] else x)
                if limit_value[1] != None:
                    train_temp[col] = train_temp[col].apply(
                        lambda x: x*(1+treatment_change_value) if x*(1+treatment_change_value) < limit_value[1] else x)
            outcome_pd[col] = xgb.predict(train_temp[feature_columns])
    result["treatment_cont_common"] = treatment_cont_common

    # 基于类别型变量的处理
    treatment_cate_common = dict()
    for col in treatment_columns_category:
        train_temp = train.copy(deep=True)
        if col in treatment_columns_common:
            pd_category = pd.DataFrame()
            print("{} is category 变量，在common中 ".format(col))
            treat_enum = treatment_columns_category_dict[col]
            print(treat_enum)
            for enum_value in treat_enum.keys():
                print(enum_value)
                train_temp[col] = enum_value
                pd_category[enum_value] = xgb.predict(
                    train_temp[feature_columns])
            d = dict(pd_category.mean())
            d = sorted(d.items(), key=lambda x: x[1], reverse=True)[0][0]
            treatment_cate_common[col] = treat_enum[d]
            print("\n\n")
        else:
            pd_category = pd.DataFrame()
            print("{} is category 变量，不在common中 ".format(col))
            treat_enum = treatment_columns_category_dict[col]
            print(treat_enum)
            for enum_value in treat_enum.keys():
                print(enum_value)
                train_temp[col] = enum_value
                pd_category[enum_value] = xgb.predict(
                    train_temp[feature_columns])
            print("\n\n")
            cate_max = pd_category.max(axis=1)
            cate_idxmax = pd_category.idxmax(axis=1)
            pd_category[col] = cate_max
            pd_category[col+"_idx"] = cate_idxmax

            outcome_pd[col] = cate_max
            outcome_pd[col+"_idx"] = cate_idxmax

    result["treatment_cate_common"] = treatment_cate_common

    temp_pd = outcome_pd[[i for i in treatment_columns_category+list(
        treatment_columns_continuous.keys()) if i not in treatment_columns_common]]
    outcome_pd['max_idx'] = temp_pd.idxmax(axis=1)
    outcome_pd["user_id"] = train["user_id"].values
    user_res = dict()
    outcome_pd_cols = outcome_pd.columns.to_list()
    for key, value in outcome_pd.iterrows():
        max_idx = value["max_idx"]
        if max_idx+"_idx" in outcome_pd_cols:
            temp = dict()
            temp[max_idx] = value[max_idx+"_idx"]
            user_res[value["user_id"]] = temp
        else:
            user_res[value["user_id"]] = max_idx
    #     print(max_idx)
    result["user_treatment"] = user_res

    # 保存文件
    with open("causal_inference.pkl", "wb") as tf:
        pickle.dump(result, tf)

    return result

In [71]:
m=inference(inference_data, treatment_columns_category_dict, xgb, feature_columns,
              treatment_columns_category, treatment_columns_common, outcome_column, treatment_columns_continuous)

V_10 is category 变量，不在common中 
{0: 'no', 1: 'unknown', 2: 'yes'}
0
1
2



treatment is category 变量，在common中 
{0: 0, 1: 1, 2: 2}
0
1
2





In [99]:
train = pd.read_csv('train.csv')
train = train[['V_0', 'V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9',
              'V_10', 'treatment', 'outcome']]
train['user_id']=train.index

feature_columns=['V_0', 'V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9',
       'V_10', 'treatment']
# treatment_columns_continuous=['V_6', 'V_7']
treatment_columns_category=['V_10', 'treatment']
treatment_columns_common=['V_6','treatment']
treatment_change_value=0.2

outcome_column=["outcome"]
treatment_columns_continuous={
  "V_6": [None, 10, 1],
  "V_7": [0.3, None, 0]
}

train_data=train[0:30000]
inference_data=train[30000:]

In [101]:
inference_data.to_csv("inference_data.csv")

In [92]:
import argparse
# 1.创建解释器
parser = argparse.ArgumentParser()

In [93]:
parser.add_argument('--train_data_path', type=str, default="train_data",help="训练文件路径")
parser.add_argument('--inference_data_path', type=str, default="inference_data",help="推理文件路径")
parser.add_argument('--feature_columns', type=str, default="",help="所有的特征list")

parser.add_argument('--treatment_columns_category', type=str, default="",help="枚举值的treatment，list")
parser.add_argument('--treatment_columns_continuous', type=str, default="",help="连续值的treatment，dict")

parser.add_argument('--treatment_columns_common', type=str, default="",help="预先定义的treatment，list")
parser.add_argument('--outcome_column', type=str, default="",help="标签列名，list")
parser.add_argument('--userid_column', type=str, default="user_id",help="userid列名，list")
args = parser.parse_args()

_StoreAction(option_strings=['--userid_column'], dest='userid_column', nargs=None, const=None, default='user_id', type=<class 'str'>, choices=None, help='userid列名，list', metavar=None)

In [98]:
eval??

In [75]:
a="['V_0', 'V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9','V_10', 'treatment']"
a="{'V_6': [None, 10, 1],'V_7': [0.3, None, 0]}"